In [1]:
# import modules
import pandas as pd

In [2]:
# declare files to load
school_data_to_load = 'Resources/schools_complete.csv'
student_data_to_load = 'Resources/students_complete.csv'

In [3]:
# read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load, encoding='utf-8')
student_data = pd.read_csv(student_data_to_load, encoding='utf-8')

In [4]:
# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

In [5]:
# prepare `District Summary` data

# retrieve the list of school names (unique values only)
list_of_schools = school_data_complete['school_name'].unique()

# calculate how many schools are in the district 
districtSchools = len(list_of_schools)

# calculate the total number of students in the district
districtStudents = len(school_data_complete)

# calculate the total budget for the district
districtBudget = school_data_complete[['school_name','budget']].drop_duplicates()['budget'].sum()

# calculate the average Math score in the district
districtAvgMath = school_data_complete['math_score'].mean()

# average reading score
districtAvgRead = school_data_complete['reading_score'].mean()

In [6]:
# declare the passing score
passingScore = 70

cond = (school_data_complete['math_score'] >= passingScore)
districtPctPassMath = (len(school_data_complete[cond])/districtStudents) * 100

# calculate percent of passing reading score in the district
cond = (school_data_complete['reading_score'] >= passingScore)
districtPctPassRead = (len(school_data_complete[cond])/districtStudents) * 100

# calculate percent of overall passing rate in the district
districtPctPassRate = (districtPctPassMath+districtPctPassRead)/2.0

## District Summary

In [7]:
# prepare `District Summary` table

# declare column labels
columns = [
    'Total Schools',
    'Total Students',
    'Total Budget',
    'Average Math Score',
    'Average Reading Score',
    '% Passing Math',
    '% Passing Reading',
    '% Overall Passing Rate'
]

# declare table data
data = [[
    districtSchools,
    districtStudents,
    districtBudget,
    districtAvgMath,
    districtAvgRead,
    districtPctPassMath,
    districtPctPassRead,
    (districtPctPassMath+districtPctPassRead)/2
]]


In [8]:
# create table
district = pd.DataFrame(data, columns=columns)

# display the `District Summary` table
# display `Total Students` values with commas
# display `Total Budget` values as currency
(district.style
         .format({ 'Total Students': '{:,}', 'Total Budget': '${:,.2f}' })
         .hide_index())

Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
15,"39,170","$24,649,428.00",78.9854,81.8778,74.9809,85.8055,80.3932


In [9]:
# prepare `Schools Summary` data

# create table for per school calculations
schools_summary = pd.DataFrame(index=list_of_schools)

# retrieve school type by school
schools_summary['School Type'] = school_data_complete.groupby('school_name')['type'].first()

# calculate number of students in each school
schools_summary['Total Students'] = school_data_complete.groupby('school_name')['student_name'].count()

# retrieve each school budget
schools_summary['Total School Budget'] = school_data_complete[['school_name','budget']].drop_duplicates().groupby('school_name')['budget'].sum()

# calculate the per student budget by school
schools_summary['Per Student Budget'] = school_data_complete[['school_name','budget']].drop_duplicates().groupby('school_name')['budget'].sum() / school_data_complete.groupby('school_name')['student_name'].count()

# calculate the average Math score by school
schools_summary['Average Math Score'] = school_data_complete.groupby('school_name')['math_score'].mean()

# calculate the average Reading score by school
schools_summary['Average Reading Score'] = school_data_complete.groupby('school_name')['reading_score'].mean()


In [10]:
# declare passing score
passingScore = 70

# declare math score passing condition
cond = (school_data_complete['math_score'] >= passingScore)

# calculate percent of passing math score
schools_summary['% Passing Math'] = school_data_complete[cond].groupby('school_name')['math_score'].count() / school_data_complete.groupby('school_name')['student_name'].count() * 100

# declare reading score passing condition
cond = (school_data_complete['reading_score'] >= passingScore)

# calculate percent of passing reading score
schools_summary['% Passing Reading'] = school_data_complete[cond].groupby('school_name')['reading_score'].count() / school_data_complete.groupby('school_name')['student_name'].count() * 100

# calculate percent of overall passing rate
schools_summary['% Overall Passing Rate'] = (schools_summary['% Passing Math']+schools_summary['% Passing Reading'])/2


## Top Performing Schools (By Passing Rate)

In [36]:
# retrieve table of top schools sorted in decending order by Passing Rate
top_schools = schools_summary.sort_values('% Overall Passing Rate', ascending=False)

# display first 5 rows of table 
# with values rounded to 1 decimal placement
# display `Total Students` values with commas
# display `Total School Budget` values as currency
# display `Per Student Budget` values as currency
(top_schools.head()
            .round(1)
            .style
            .format({
                'Total Students': '{:,}',
                'Total School Budget': '${:,.2f}', 
                'Per Student Budget': '${:,.2f}' 
            }))

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.1,84,94.1,97,95.6
Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.4,83.8,93.3,97.3,95.3
Pena High School,Charter,962,"$585,858.00",$609.00,83.8,84,94.6,95.9,95.3
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.4,83.8,93.4,97.1,95.3
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.3,84,93.9,96.5,95.2


## Bottom Performing Schools (By Passing Rate)

In [33]:
# retrieve table of top schools sorted in ascending order by Passing Rate
bottom_schools = schools_summary.sort_values('% Overall Passing Rate', ascending=True)

# display first 5 rows of table 
# with values rounded to 1 decimal placement
# display `Total Students` values with commas
# display `Total School Budget` values as currency
# display `Per Student Budget` values as currency
(bottom_schools.head()
            .round(1)
            .style
            .format({
                'Total Students': '{:,}',
                'Total School Budget': '${:,.2f}', 
                'Per Student Budget': '${:,.2f}' 
            }))


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.8,80.7,66.4,80.2,73.3
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.7,81.2,66,80.7,73.4
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.6,81.2,65.7,81.3,73.5
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.1,81,66.1,81.2,73.6
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.1,80.7,68.3,79.3,73.8


In [27]:
# get a sorted list of the school grades
grades = sorted(list(school_data_complete['grade'].unique()),key=lambda x: int( x.split('th')[0] ))

## Math Scores by Grade

In [37]:
# create table
math_by_grade = pd.DataFrame(index=list_of_schools, columns=grades)

# for each grade calculate the average Math score
for grade in grades:

    # define condition to get data for specific grade
    cond = (school_data_complete['grade'] == grade)
    
    # calculate average Math score for grade matching condition
    # rounded to 1 decimal placement
    math_by_grade[grade] = school_data_complete[cond].groupby('school_name')['math_score'].mean()

# sort schools in ascending alphabetical order
math_by_grade = math_by_grade.sort_index()

# display table with values rounded to 1 decimal placement
math_by_grade.round(1)

,9th,10th,11th,12th
Bailey High School,77.1,77.0,77.5,76.5
Cabrera High School,83.1,83.2,82.8,83.3
Figueroa High School,76.4,76.5,76.9,77.2
Ford High School,77.4,77.7,76.9,76.2
Griffin High School,82.0,84.2,83.8,83.4
Hernandez High School,77.4,77.3,77.1,77.2
Holden High School,83.8,83.4,85.0,82.9
Huang High School,77.0,75.9,76.4,77.2
Johnson High School,77.2,76.7,77.5,76.9
Pena High School,83.6,83.4,84.3,84.1


## Reading Score by Grade

In [38]:
# create table
read_by_grade = pd.DataFrame(index=list_of_schools, columns=grades)

# for each grade calculate the average Reading score
for grade in grades:

    # define condition to get data for specific grade
    cond = (school_data_complete['grade'] == grade)
    
    # calculate average Reading score for grade matching condition
    # rounded to 1 decimal placement
    read_by_grade[grade] = school_data_complete[cond].groupby('school_name')['reading_score'].mean()

# sort schools in ascending alphabetical order
read_by_grade = read_by_grade.sort_index()

# display table with values rounded to 1 decimal placement
read_by_grade.round(1)

,9th,10th,11th,12th
Bailey High School,81.3,80.9,80.9,80.9
Cabrera High School,83.7,84.3,83.8,84.3
Figueroa High School,81.2,81.4,80.6,81.4
Ford High School,80.6,81.3,80.4,80.7
Griffin High School,83.4,83.7,84.3,84.0
Hernandez High School,80.9,80.7,81.4,80.9
Holden High School,83.7,83.3,83.8,84.7
Huang High School,81.3,81.5,81.4,80.3
Johnson High School,81.3,80.8,80.6,81.2
Pena High School,83.8,83.6,84.3,84.6


## Scores by School Spending

In [39]:
# group spending by per student budget

# declare the spending ranges
spending_bins = [0, 585, 615, 645, 675]

# declare the labels for the spending ranges
group_names = ['<$585', '$585-615', '$615-645', '$645-675']

# create table
schools_spending = pd.DataFrame()

# copy schools_summary table with no index
temp = schools_summary.copy().reset_index()

# segment the `Per Student Budget`
temp['Spending Ranges (Per Student)'] = pd.cut(temp["Per Student Budget"], spending_bins, labels = group_names,  include_lowest=True)

# calculate the average Math score by spending range
schools_spending['Average Math Score'] = temp.groupby('Spending Ranges (Per Student)')['Average Math Score'].mean()

# calculate the average Reading score by spending range
schools_spending['Average Reading Score'] = temp.groupby('Spending Ranges (Per Student)')['Average Reading Score'].mean()

# calculate the rate of passing Math score by spending range
schools_spending['% Passing Math'] = temp.groupby('Spending Ranges (Per Student)')['% Passing Math'].mean()

# calculate the rate of passing Reading score by spending range
schools_spending['% Passing Reading'] = temp.groupby('Spending Ranges (Per Student)')['% Passing Reading'].mean()

# calculate the overall passing rate by spending range
schools_spending['% Overall Passing Rate'] = temp.groupby('Spending Ranges (Per Student)')['% Overall Passing Rate'].mean()

# display table with values rounded to 1 decimal placement
schools_spending.round(1)

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.5,83.9,93.5,96.6,95.0
$585-615,83.6,83.9,94.2,95.9,95.1
$615-645,79.1,81.9,75.7,86.1,80.9
$645-675,77.0,81.0,66.2,81.1,73.6


## Scores by School Size

In [40]:
# group by school size

# declare school size ranges
size_bins = [0, 1000, 2000, 5000]

# declare school size labels
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

# create table
schools_size = pd.DataFrame()

# copy schools_summary table with no index
temp = schools_summary.copy().reset_index()

# segment by `Total Students`
temp['School Size'] = pd.cut(temp['Total Students'], size_bins, labels = group_names,  include_lowest=True)

# calculate the average Math score by school size
schools_size['Average Math Score'] = temp.groupby('School Size')['Average Math Score'].mean()

# calculate the average Reading score by school size
schools_size['Average Reading Score'] = temp.groupby('School Size')['Average Reading Score'].mean()

# calculate the rate of passing Math score by school size
schools_size['% Passing Math'] = temp.groupby('School Size')['% Passing Math'].mean()

# calculate the rate of passing Reading score by school size
schools_size['% Passing Reading'] = temp.groupby('School Size')['% Passing Reading'].mean()

# calculate the overall rate of passing by school size
schools_size['% Overall Passing Rate'] = temp.groupby('School Size')['% Overall Passing Rate'].mean()

# display table with values rounded to 1 decimal placement
schools_size.round(1)


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.8,83.9,93.6,96.1,94.8
Medium (1000-2000),83.4,83.9,93.6,96.8,95.2
Large (2000-5000),77.7,81.3,70.0,82.8,76.4


## Scores by School Type

In [41]:
# create table
schools_type = pd.DataFrame()

# copy schools_summary table with no index
temp = schools_summary.copy().reset_index()

# calculate the average Math score by school type
schools_type['Average Math Score'] = temp.groupby('School Type')['Average Math Score'].mean()

# calculate the average Reading score by school type
schools_type['Average Reading Score'] = temp.groupby('School Type')['Average Reading Score'].mean()

# calculate the rate of passing Math score by school type
schools_type['% Passing Math'] = temp.groupby('School Type')['% Passing Math'].mean()

# calculate the rate of passing Reading score by school type
schools_type['% Passing Reading'] = temp.groupby('School Type')['% Passing Reading'].mean()

# calculate the overall rate of passing by school type
schools_type['% Overall Passing Rate'] = temp.groupby('School Type')['% Overall Passing Rate'].mean()

# display table with values rounded to 1 decimal placement
schools_type.round(1)

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.5,83.9,93.6,96.6,95.1
District,77.0,81.0,66.5,80.8,73.7
